In [12]:
from bert_logistic import prepare_data_for_model, read_texts_from_dir, train_and_evaluate
import pandas as pd
import numpy as np

In [13]:
train_path = "/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/train"
test_path = "/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/test"
gt_path = "/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/train.csv"
    

In [ ]:
print("Loading data...")
df_train = read_texts_from_dir(train_path)
df_test = read_texts_from_dir(test_path)
df_train_gt = pd.read_csv(gt_path)
y_train = df_train_gt["real_text_id"].values
    

Loading data...
Number of directories: 95
Number of directories: 1068


In [25]:
full_df_train = pd.concat([df_train, df_train_gt["real_text_id"]], axis=1)
full_df_train.head()

,file_1,file_2,real_text_id
0,The VIRSA (Visible Infrared Survey Telescope A...,The China relay network has released a signifi...,1
1,China\nThe goal of this project involves achie...,The project aims to achieve an accuracy level ...,2
2,Scientists can learn about how galaxies form a...,Dinosaur eggshells offer clues about what dino...,1
3,China\nThe study suggests that multiple star s...,The importance for understanding how stars evo...,2
4,Dinosaur Rex was excited about his new toy set...,Analyzing how fast stars rotate within a galax...,2


In [30]:
# chọn ra random 20 hàng từ full_df_train
df_train_sample = full_df_train.sample(n=80, random_state=42)
df_train_sample.reset_index(drop=True, inplace=True)
df_train_sample.to_json("df_train_sample.json", orient="records", lines=True)

In [15]:
print("Preparing training data...")
X_train, embedding_extractor = prepare_data_for_model(
    df_train, 
    fit_embedding=True, 
    model_name='intfloat/multilingual-e5-small'
)

Preparing training data...
Step 1: Preprocessing text...


Cleaning text: 100%|██████████| 95/95 [00:00<00:00, 1028.79it/s]


Step 2: Extracting rule-based features...


Extracting rule-based features: 100%|██████████| 95/95 [00:00<00:00, 200.43it/s]


Step 3: Extracting statistical features...
Step 4: Extracting embedding features...
Loading embedding model: intfloat/multilingual-e5-small
Loaded as SentenceTransformer model
Extracting embedding features for file_1...


Extracting embeddings: 100%|██████████| 3/3 [00:08<00:00,  2.80s/it]


Extracting embedding features for file_2...


Extracting embeddings: 100%|██████████| 3/3 [00:07<00:00,  2.61s/it]

Step 5: Combining all features...
Final feature matrix shape: (95, 1580)


In [16]:
X_test, _ = prepare_data_for_model(
        df_test, 
        embedding_extractor=embedding_extractor, 
        fit_embedding=False
    )

Step 1: Preprocessing text...


Cleaning text: 100%|██████████| 1068/1068 [00:01<00:00, 857.87it/s]


Step 2: Extracting rule-based features...


Extracting rule-based features: 100%|██████████| 1068/1068 [00:04<00:00, 230.40it/s]


Step 3: Extracting statistical features...
Step 4: Extracting embedding features...
Extracting embedding features for file_1...


Extracting embeddings: 100%|██████████| 34/34 [01:25<00:00,  2.51s/it]


Extracting embedding features for file_2...


Extracting embeddings: 100%|██████████| 34/34 [01:29<00:00,  2.64s/it]

Step 5: Combining all features...
Final feature matrix shape: (1068, 1580)


In [17]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [20]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

# Initialize CatBoostClassifier
catboost_model = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=6, verbose=0)

# Train the model
catboost_model.fit(X_train, y_train)

# Predict on validation set
y_preds_val_catboost = catboost_model.predict(X_val)

# Calculate accuracy
acc_catboost = accuracy_score(y_val, y_preds_val_catboost)
print(f"Validation accuracy with CatBoost: {acc_catboost:.4f}")

Validation accuracy with CatBoost: 1.0000


In [21]:
from pathlib import Path

print("Predicting on test ...")
test_pred = catboost_model.predict(X_test)

# --- Build submission -------------------------------------------------
submission = pd.DataFrame({
    "id": df_test.index,
    "real_text_id": test_pred.astype(int)
}).sort_values("id")

save_path = Path("submission_e5_catboost.csv")
submission.to_csv(save_path, index=False)
print(f"✅ Submission saved to {save_path.resolve()}")

Predicting on test ...
✅ Submission saved to /home/thangquang09/CODE/CTAI_MachineLearning/notebooks/submission_e5_catboost.csv
